# 판매자 피쳐 엔지니어링

In [1]:
import pandas as pd
from pathlib import Path

OUTPUT_DIR = Path('../data/processed')
OUTPUT_DIR.mkdir(exist_ok=True)

In [2]:
# 1. 파일 로드
csv_path = OUTPUT_DIR / 'sellers_ml_ready.csv'

try:
    df = pd.read_csv(csv_path)
except FileNotFoundError:
    print("❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
    exit()

In [3]:
# 2. 상위 15개 지역 확인 및 출력
top_15_addresses = df['city_state'].value_counts().nlargest(15).index.tolist()

print("\n" + "="*50)
print("📍 [기획자 확인용] 우리 플랫폼 판매자 밀집 TOP 15 지역")
print("-"*50)
for i, addr in enumerate(top_15_addresses, 1):
    count = df['city_state'].value_counts()[addr]
    print(f"{i}위: {addr} ({count}명)")
print("="*50 + "\n")


📍 [기획자 확인용] 우리 플랫폼 판매자 밀집 TOP 15 지역
--------------------------------------------------
1위: 경기도 김포시 (24명)
2위: 경기도 남양주시 (21명)
3위: 경기도 용인시 (16명)
4위: 경기도 고양시 (15명)
5위: 서울특별시 용산구 (12명)
6위: 경기도 안양시 (9명)
7위: 서울특별시 송파구 (8명)
8위: 인천광역시 서구 (8명)
9위: 인천광역시 연수구 (8명)
10위: 서울특별시 금천구 (7명)
11위: 경기도 파주시 (7명)
12위: 경기도 부천시 (7명)
13위: 서울특별시 강남구 (7명)
14위: 서울특별시 구로구 (7명)
15위: 경기도 안산시 (6명)



In [4]:
# 3. 원-핫 인코딩 전략 실행
# 상위 15개 외에는 'others'로 통합
df['address_refined'] = df['city_state'].apply(lambda x: x if x in top_15_addresses else 'others')

# 원-핫 인코딩 수행
df_final = pd.get_dummies(df, columns=['address_refined'], prefix='addr')


In [5]:
# 4. 불필요한 텍스트 컬럼 및 중복 컬럼 제거
# 모델은 숫자만 읽어야 하므로 텍스트인 city_state는 삭제합니다.
if 'city_state' in df_final.columns:
    df_final = df_final.drop(columns=['city_state'])


In [6]:
# 5. 최종 결과 저장
output_path = OUTPUT_DIR / 'sellers_final_for_ml.csv'
df_final.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ 머신러닝용 최종 데이터셋 생성 완료!")
print(f"📂 저장 경로: {output_path}")
print(f"📊 최종 컬럼 수: {len(df_final.columns)}개")

✅ 머신러닝용 최종 데이터셋 생성 완료!
📂 저장 경로: ..\data\processed\sellers_final_for_ml.csv
📊 최종 컬럼 수: 24개
